In [1]:
max_features = 128
maxlen = 128
epochs = 20

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
json_list = list()
with open('../data/tweets_DM.json' , 'r') as file:
    for line in file:
        json_list.append(json.loads(line))

In [5]:
tweet_list = list()
for json in json_list:
    tweet_id = json['_source']['tweet']['tweet_id']
    hashtags = json['_source']['tweet']['hashtags']
    hashtag = ' '.join(hashtags)
    text = json['_source']['tweet']['text']
    text = text + ' ' + hashtag
    tweet_list.append([tweet_id, text])

In [6]:
text_df = pd.DataFrame(tweet_list, columns=['tweet_id', 'text'])

In [7]:
emotion_df = pd.read_csv('../data/emotion.csv')
emotion_df

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [8]:
identification_df = pd.read_csv('../data/data_identification.csv')
identification_df

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [9]:
test_df = identification_df[identification_df['identification'] == 'test']
test_df

,tweet_id,identification
0,0x28cc61,test
3,0x2db41f,test
15,0x2466f6,test
23,0x23f9e9,test
31,0x1fb4e1,test
...,...,...
1867495,0x2c4dc2,test
1867496,0x31be7c,test
1867500,0x1ca58e,test
1867515,0x35c8ba,test


In [10]:
train_df = text_df.merge(emotion_df, left_on='tweet_id', right_on='tweet_id')
train_df

,tweet_id,text,emotion
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation
...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy
1455559,0x38959e,In every circumtance I'd like to be thankful t...,joy
1455560,0x2cbca6,there's currently two girls walking around the...,joy
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",joy


In [11]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['emotion'])
train_df['label'] = le.transform(train_df['emotion'])
train_df

,tweet_id,text,emotion,label
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",anticipation,1
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",sadness,5
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,fear,3
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,joy,4
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,anticipation,1
...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,joy,4
1455559,0x38959e,In every circumtance I'd like to be thankful t...,joy,4
1455560,0x2cbca6,there's currently two girls walking around the...,joy,4
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",joy,4


In [12]:
test_df = test_df.merge(text_df, left_on='tweet_id', right_on='tweet_id')
test_df

,tweet_id,identification,text
0,0x28cc61,test,@Habbo I've seen two separate colours of the e...
1,0x2db41f,test,@FoxNews @KellyannePolls No serious self respe...
2,0x2466f6,test,"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,test,@cineworld “only the brave” just out and fount...
4,0x1fb4e1,test,Felt like total dog 💩 going into open gym and ...
...,...,...,...
411967,0x2c4dc2,test,6 year old walks in astounded. Mum! Look how b...
411968,0x31be7c,test,Only one week to go until the #inspiringvolunt...
411969,0x1ca58e,test,"I just got caught up with the manga for ""My He..."
411970,0x35c8ba,test,Speak only when spoken to and make hot ass mus...


In [13]:
pretrained_path = '/home/Danny/Data-Mining/lab2/kaggle/embedding/bert_tiny'
config_path = pretrained_path + '/bert_config.json'
checkpoint_path = pretrained_path + '/bert_model.ckpt'
dict_path = pretrained_path + '/vocab.txt'

In [14]:
import codecs
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

token_dict = dict()

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

In [15]:
x_array = train_df['text']
x_array.shape

(1455563,)

In [16]:
from keras.utils import to_categorical 
y_array = train_df['label'].to_numpy()
# y_array = to_categorical(y_array)
y_array.shape

(1455563,)

In [17]:
data = list()
for x, y in zip(x_array, y_array):
    data.append((x, y))

In [18]:
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]
print(len(train_data))
print(len(valid_data))

1310006
145557


In [19]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []

In [20]:
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='softmax')(x)

model = Model([x1_in, x2_in], p)
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, None, 128)    4369408     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128)          0           model_1[1][0]              

In [21]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)

In [22]:
model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=epochs,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
40938/40938 [==============================] - 1675s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00 - val_accuracy: 0.1704
Epoch 2/20
40938/40938 [==============================] - 1668s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00 - val_accuracy: 0.1704
Epoch 3/20
40938/40938 [==============================] - 1676s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00 - val_accuracy: 0.1705
Epoch 4/20
40938/40938 [==============================] - 1675s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00 - val_accuracy: 0.1704
Epoch 5/20
40938/40938 [==============================] - 1668s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00 - val_accuracy: 0.1705
Epoch 6/20
40938/40938 [==============================] - 1675s 41ms/step - loss: 0.0000e+00 - accuracy: 0.1711 - val_loss: 0.0000e+00

In [23]:
model_path = 'model/keras_bert_len_{}_epo_{}'.format(max_features, epochs)
model.save(model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/keras_bert_len_128_epo_20/assets


In [24]:
from tensorflow import keras
model = keras.models.load_model(model_path)
model.evaluate(x_test, y_test)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * Tensor("inputs:0", shape=(None, None, 128), dtype=float32)
  Keyword arguments: {'training': True}

Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, None, 128), dtype=tf.float32, name='inputs')
  Keyword arguments: {'training': True, 'mask': TensorSpec(shape=(None, None), dtype=tf.bool, name='mask')}

Option 2:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, None, 128), dtype=tf.float32, name='inputs')
  Keyword arguments: {'training': False, 'mask': TensorSpec(shape=(None, None), dtype=tf.bool, name='mask')}

In [ ]:
y_pred = model.predict_classes(x_test)
y_pred.shape

In [ ]:
y_test = np.argmax(y_test, axis=1)
y_test.shape

In [ ]:
label_count = len(pd.unique(train_df['label']))
target_names =  le.inverse_transform([i for i in range(label_count)])
target_names

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
x_array = word2vec(test_df['text'], maxlen)
x_array.shape

In [ ]:
y_pred = model.predict_classes(x_array, batch_size=10)
y_pred.shape

In [ ]:
predict_array =  le.inverse_transform(y_pred)
predict_array.shape

In [ ]:
test_df['predict'] = predict_array
test_df

In [ ]:
output_df = test_df[['tweet_id', 'predict']]
output_df = output_df.rename(columns={'tweet_id':'id', 'predict':'emotion'})
output_df

In [ ]:
output_path = '../../output/keras_bert_len_{}_epo_{}.csv'.format(max_features, epochs)
output_df.to_csv(output_path, index=False, header=True)